In [ ]:
import dataAck
import portfolio
allTickers = dataAck.getAllTickersPlain()
import params

In [ ]:
while True:
    import random
    tickerToTrade = allTickers[random.randint(0, len(allTickers)) - 1]
    print(tickerToTrade)
    
    tData = dataAck.getTrainingData(tickerToTrade)
    joinedData = None
    validTickers = None

    
    
    if tData is None:
        dataAck.logModel("Cache", {
            "type":"miss",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })

        tickersToPull = dataAck.getDataSourcesForTicker(tickerToTrade)
        print(tickersToPull)

        pulledData, validTickers = dataAck.downloadTickerData(tickersToPull)

        joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
        
        dataAck.storeTrainingData(tickerToTrade, (joinedData, validTickers))
    else:
        joinedData = tData[0]
        validTickers = tData[1]
        dataAck.logModel("Cache", {
            "type":"hit",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })
        
    
    sManager = dataAck.seriesManager(validTickers)
    print(sManager.describe())

    import time
    import warnings
    import numpy as np
    warnings.filterwarnings("ignore")
    ##GET ALGOS INITIALLY GOOD
    runsSeen = 0
    while True:
        s = sManager.createSeries()
        while s.checkValidity(s.transformJoinedData(joinedData[:"2016-01-01"])) == False:
            s = sManager.createSeries()
            

        try:
            for defaultWindowSize in [5, 10, 22, 44]:
                for trees in [25, 50, 100, 150, 300]:
                    for predictionLength in [3, 5, 7, 10]:
                        if random.uniform(0,1) < 0.5: ##RANDOMLY SKIP
                            continue
                        b = dataAck.algoBlob(s, defaultWindowSize, trees, predictionLength, tickerToTrade)
                        algoReturn, factorReturn, predictions =  b.makePredictions(portfolio.prepareDataForModel(b, joinedData), daysToCheck = None, earlyStop = True)
                        if algoReturn is None:
                            toLog = factorReturn
                            if np.isnan(toLog["sharpe"]) == True:
                                raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                                
                            toLog["modelDescription"] = str(b.describe())
                            dataAck.logModel("Model Stopped Early", toLog)
                            print("Model Stopped Early", toLog)
                            continue
                        metrics = dataAck.vizResults(algoReturn[:-252], factorReturn[:-252], False)
                        print("TRAIN:", metrics)
                        if np.isnan(metrics["SHARPE"]) == True:
                            raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                        
                        if metrics["SHARPE"] > 1.0 and metrics["ACTIVITY"] > 0.7 and metrics["BETA"] < 0.6 and metrics["TREYNOR"] > 0.0:
                            ##STORE
                            testMetrics = dataAck.vizResults(algoReturn[-252:], factorReturn[-252:], False)
                            print("TEST:", testMetrics)
                            print("TODAY:", b.makeTodayPrediction(portfolio.prepareDataForModel(b, joinedData)))
                            dataAck.storeModelData(b, algoReturn, predictions)
                            dataAck.storeModel(b, metrics, testMetrics)
                        else:
                            dataAck.logModel("Model Skipped", {
                                "modelDescription":str(b.describe()),
                                "sharpe":metrics["SHARPE"], ##OVERLOADED IN FAIL
                                "activity":metrics["ACTIVITY"],
                                "beta":metrics["BETA"]
                            })
        except:
            print("FAILED", s.describe())
            dataAck.logModel("Series Failed", {
                "seriesDescription":str(s.describe())
            })

        runsSeen += 1

        if runsSeen > 10:
            ##START NEW TICKER
            dataAck.logModel("Search Update", {
                "message":"restarting search with different ticker",
                "currentTicker":tickerToTrade
            })
            break


IVV
ATTEMPTING PULL IVV
['AGG', 'DXJ', 'EWJ', 'FXE', 'GDX', 'GLD', 'IAU', 'IEF', 'LQD', 'SHY', 'SLV', 'TIP', 'TLT', 'USO', 'XLP', 'XLU', 'XLV', 'IVV']
(5, ('IVV', 6, None, None, 3), 3, 'IVV')
SERIES (5, ('IVV', 6, None, None, 3), 3, 'IVV') WINDOW 5 TREES 25
FAILED ('IVV', 6, None, None, 3)
(5, ('IVV', 12, None, 9, 2), 3, 'IVV')
SERIES (5, ('IVV', 12, None, 9, 2), 3, 'IVV') WINDOW 5 TREES 25
FAILED ('IVV', 12, None, 9, 2)
(5, ('GLD', 36, None, None, 1), 3, 'IVV')
SERIES (5, ('GLD', 36, None, None, 1), 3, 'IVV') WINDOW 5 TREES 25
CONTINUING SHARPE: 0.328008599157 BETA: 0.211296576562 TREYNOR: 0.537276868451
Model Stopped Early {'sharpe': -1.3536205803848025, 'beta': 0.21250004773925002, 'activity': 0.695, 'treynor': -0.71124145988009246, 'period': 'first 600 days', 'modelDescription': "((5, ('GLD', 36, None, None, 1), 3, 'IVV'), 5, 25)"}
(5, ('GLD', 36, None, None, 1), 5, 'IVV')
SERIES (5, ('GLD', 36, None, None, 1), 5, 'IVV') WINDOW 5 TREES 50
CONTINUING SHARPE: 0.128969443716 BETA: 0.2

Process ForkProcess-151:
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 309, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 295, in runDay
    pred = totalModel.predict_proba(np.array([xTarget]))[0][1]
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py", line 583, in predict_proba
    for e in self.estimators_)
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 758, in __call__
    while self.dispatch_one_batch(iterator):
  File "/home/prohb125/anaconda3/lib/python3.6/site-packages/sklearn/externals/j

FAILED ('GLD', 36, None, None, 1)
(5, ('GLD', 31, None, None, 1), 3, 'IVV')
SERIES (5, ('GLD', 31, None, None, 1), 3, 'IVV') WINDOW 5 TREES 25
FAILED ('GLD', 31, None, None, 1)
(5, ('TLT', 24, 19, 14, 2), 3, 'IVV')
SERIES (5, ('TLT', 24, 19, 14, 2), 3, 'IVV') WINDOW 5 TREES 25
Model Stopped Early {'sharpe': 0.81290497108107396, 'beta': 0.99808394712613946, 'activity': 0.9920634920634921, 'treynor': -0.0039401014644900242, 'period': 'first 252 days', 'modelDescription': "((5, ('TLT', 24, 19, 14, 2), 3, 'IVV'), 5, 25)"}
(5, ('TLT', 24, 19, 14, 2), 7, 'IVV')
SERIES (5, ('TLT', 24, 19, 14, 2), 7, 'IVV') WINDOW 5 TREES 25
